In [1]:
import numpy as np
import pandas as pd

import matplotlib as mpl
import matplotlib.pyplot as plt
from collections import Counter
from pyspark.sql import SparkSession
from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.ml.recommendation import ALS
from pyspark.sql import Row
from pyspark.ml.tuning import ParamGridBuilder, CrossValidator
from numpy.linalg import norm
import time
from pyspark.sql.functions import split, explode,col
from pyspark.ml.feature import StringIndexer
from pyspark.ml import Pipeline


In [2]:
line_item=['x'+str(i) for i in range(15)]
import random  
import string  
def specific_string(length):  
    sample_string = 'pw y' # define teh specific string  
    # define teh condition for random string  
    result = ''.join((random.choice(sample_string)) for x in range(length))  
    #print(" Randomly generated string is: ", result)  
    return result
KW=[]
for i in range(len(line_item)):
    KW.append([])
    for j in range(5):
        KW[i].append(specific_string(np.random.randint(2,5)))
def specific_user_segment():  
    sample_string = 'segment' # define teh specific string
    idx=np.random.randint(0,50)
    return sample_string+str(idx)
US=[]
for i in range(len(line_item)):
    US.append([])
    for j in range(3):
        US[i].append(specific_user_segment())
advertiser_id=[np.random.randint(0,6) for _ in range(len(line_item))]
camp_category=['food']*10+['electric']*5
df_kw=pd.DataFrame({
    'line_item_id':line_item,
    'keywords':KW,
    'advertiser_id':advertiser_id,
    'camp_category':camp_category
})
df_kw_exp=df_kw.explode('keywords')
df_us=pd.DataFrame({
    'line_item_id':line_item,
    'segments':US,
    'advertiser_id':advertiser_id,
    'camp_category':camp_category
})

df_us_exp=df_us.explode('segments')
uq_kw=list(df_kw_exp.keywords.unique())
reach=[np.random.randint(1000,5000) for _ in range(len(uq_kw))]
user_reach_KW=pd.DataFrame({
    'keywords':uq_kw,
    'user_reach':reach
})
uq_us=list(df_us_exp.segments.unique())
reach=[np.random.randint(1000,5000) for _ in range(len(uq_us))]
user_reach_US=pd.DataFrame({
    'user_segment':uq_us,
    'user_reach':reach
})

In [31]:
class Preprocess_data:
    def __init__(self,df,df_user_reach):
        self.df=df
        self.df_user_reach=df_user_reach
        
        
    def fn1(self,x):
        return (''.join([i if 32 < ord(i) < 126 else " " for i in x]))

    def fn2(self,x):
        return (''.join([i if i not in "!@#$%^&*()[]{};:,./<>?\|`~-=_+" else " " for i in x]))

    def fn3(self,x):
        return (''.join([i if i not in '!@#$%^&*()[]{};:,/<>?\|`~-=_+""\n' else " " for i in x]))
    
    def itemperse(self,d,char,dedup):
        idx=d[char]
        res=np.zeros(len(dedup))
        res[idx]=1
        return res

    def preprocessing_kw(self):
        data=list(self.df['keywords'])
        all_sentences = [l.split('\t')[0] for l in data]
        all_sentences = [self.fn1(sentence) for sentence in all_sentences]
        all_sentences = [self.fn2(sentence) for sentence in all_sentences]
        all_sentences = [sentence.lower() for sentence in all_sentences]
        all_sentences = [sentence.strip() for sentence in all_sentences]
        preprocessed_kw={data[i]:all_sentences[i] for i in range(len(data))}### user reach extraction

        self.df_user_reach['preprocessed_kw']=self.df_user_reach['keywords'].map(preprocessed_kw)
        self.df_user_reach=self.df_user_reach.groupby(['preprocessed_kw']).agg({'user_reach':'sum'}).round(2)


        self.df['kw_id']=self.df['keywords'].map(preprocessed_kw)
        df_item_user=self.df.groupby(['line_item_id','kw_id']).size().reset_index()[['line_item_id','kw_id']]
        df_item_user['rating']=1
        
        spark = SparkSession \
                    .builder \
                    .master("local") \
                    .appName("keyword targeting reco") \
                    .getOrCreate()
        sparkDF=spark.createDataFrame(df_item_user) 
        indexer = [StringIndexer(inputCol=column, outputCol=column+"_index") for column in list(set(sparkDF.columns)-set(['rating'])) ]
        pipeline = Pipeline(stages=indexer)
        transformed = pipeline.fit(sparkDF).transform(sparkDF)
        #transformed=transformed.select(['line_item_id_index','kw_id_index','rating'])

        md=transformed.select(transformed['line_item_id'],transformed['line_item_id_index'],transformed['kw_id'],transformed['kw_id_index'])
        md=md.toPandas()
        dict1 =dict(zip(md['line_item_id_index'],md['line_item_id']))
        dict2=dict(zip(md['kw_id_index'],md['kw_id']))

        return transformed,self.df_user_reach,preprocessed_kw,dict1,dict2
    
    def preprocessing_kw_cs(self):
        data=list(self.df['keywords'])
        all_sentences = [l.split('\t')[0] for l in data]
        all_sentences = [self.fn1(sentence) for sentence in all_sentences]
        all_sentences = [self.fn2(sentence) for sentence in all_sentences]
        all_sentences = [sentence.lower() for sentence in all_sentences]
        all_sentences = [sentence.strip() for sentence in all_sentences]
        preprocessed_kw={data[i]:all_sentences[i] for i in range(len(data))}### user reach extraction

        self.df_user_reach['preprocessed_kw']=self.df_user_reach['keywords'].map(preprocessed_kw)
        self.df_user_reach=self.df_user_reach.groupby(['preprocessed_kw']).agg({'user_reach':'sum'}).round(2)


        self.df['kw_id']=self.df['keywords'].map(preprocessed_kw)
        df_item_user=self.df.groupby(['advertiser_id','kw_id']).size().reset_index()[['advertiser_id','kw_id']]
        df_item_user['rating']=1
        spark = SparkSession \
                    .builder \
                    .master("local") \
                    .appName("keyword targeting reco") \
                    .getOrCreate()
        sparkDF=spark.createDataFrame(df_item_user) 
        indexer = [StringIndexer(inputCol=column, outputCol=column+"_index") for column in list(set(sparkDF.columns)-set(['rating'])) ]
        pipeline = Pipeline(stages=indexer)
        transformed = pipeline.fit(sparkDF).transform(sparkDF)
        #transformed=transformed.select(['line_item_id_index','kw_id_index','rating'])

        md=transformed.select(transformed['advertiser_id'],transformed['advertiser_id_index'],transformed['kw_id'],transformed['kw_id_index'])
        md=md.toPandas()
        dict1 =dict(zip(md['advertiser_id_index'],md['advertiser_id']))
        dict2=dict(zip(md['kw_id_index'],md['kw_id']))

        return transformed,self.df_user_reach,preprocessed_kw,dict1,dict2
    
    def preprocessing_us(self):
        df_item_user=self.df.groupby(['line_item_id','segments']).size().reset_index()[['line_item_id','segments']]
        df_item_user['rating']=1
        spark = SparkSession \
                    .builder \
                    .master("local") \
                    .appName("keyword targeting reco") \
                    .getOrCreate()
        sparkDF=spark.createDataFrame(df_item_user) 
        indexer = [StringIndexer(inputCol=column, outputCol=column+"_index") for column in list(set(sparkDF.columns)-set(['rating'])) ]
        pipeline = Pipeline(stages=indexer)
        transformed = pipeline.fit(sparkDF).transform(sparkDF)
        #transformed=transformed.select(['line_item_id_index','kw_id_index','rating'])

        md=transformed.select(transformed['line_item_id'],transformed['line_item_id_index'],transformed['segments'],transformed['segments_index'])
        md=md.toPandas()
        dict1 =dict(zip(md['line_item_id_index'],md['line_item_id']))
        dict2=dict(zip(md['segments_index'],md['segments']))
        return transformed,self.df_user_reach,dict1,dict2
    
    def preprocessing_us_cs(self):
        df_item_user=self.df.groupby(['advertiser_id','segments']).size().reset_index()[['advertiser_id','segments']]
        df_item_user['rating']=1
        spark = SparkSession \
                    .builder \
                    .master("local") \
                    .appName("keyword targeting reco") \
                    .getOrCreate()
        sparkDF=spark.createDataFrame(df_item_user) 
        indexer = [StringIndexer(inputCol=column, outputCol=column+"_index") for column in list(set(sparkDF.columns)-set(['rating'])) ]
        pipeline = Pipeline(stages=indexer)
        transformed = pipeline.fit(sparkDF).transform(sparkDF)
        #transformed=transformed.select(['line_item_id_index','kw_id_index','rating'])

        md=transformed.select(transformed['advertiser_id'],transformed['advertiser_id_index'],transformed['segments'],transformed['segments_index'])
        md=md.toPandas()
        dict1 =dict(zip(md['advertiser_id_index'],md['advertiser_id']))
        dict2=dict(zip(md['segments_index'],md['segments']))
        return transformed,self.df_user_reach,dict1,dict2

In [116]:
class targeting_tactic_model_KT:
    def __init__(self,df_tt_exp,user_reach_tt,camp_category='food'):
        self.df_tt_exp=df_tt_exp
        self.camp_category=camp_category
        self.user_reach=user_reach_tt
        spark = SparkSession \
            .builder \
            .master("local") \
            .appName("Protob Conversion to Parquet") \
            .getOrCreate()
        self.is_cold_started_trained_already=False
        self.is_base_model_trained=False
    def fit(self,line_item_id,is_needed=False):
        if not self.is_base_model_trained and line_item_id in list(self.df_tt_exp['line_item_id'].unique()) :
            self.targeting_tactic_reco_kw()
            self.is_base_model_trained=True
        if (not self.is_cold_started_trained_already) and is_needed:
            self.targeting_tactic_reco_cold_kw()
            self.is_cold_started_trained_already=True
        
    def targeting_tactic_reco_kw(self):

        self.df_tt_exp=self.df_tt_exp.loc[self.df_tt_exp['camp_category']==self.camp_category]
#         if self.line_item_id not in list(self.df_tt_exp['line_item_id'].unique()):
#             return self.targeting_tactic_reco_cold_kw()
        
        df_kwx,ur_kw,preprocessed_kw,d_lin_item,d_kw=Preprocess_data(self.df_tt_exp,self.user_reach).preprocessing_kw()
        d_reach={ur_kw.index[i]:ur_kw['user_reach'][i] for i in range(ur_kw.shape[0])}
        (training,test)=df_kwx.randomSplit([0.8, 0.2])
        als=ALS(maxIter=5,regParam=0.09,rank=25,userCol="line_item_id_index",itemCol="kw_id_index",ratingCol="rating",coldStartStrategy="drop",nonnegative=True)
        self.model=als.fit(training)

        evaluator=RegressionEvaluator(metricName="rmse",labelCol="rating",predictionCol="prediction")
        predictions=self.model.transform(test)
        rmse=evaluator.evaluate(predictions)
        print("RMSE="+str(rmse))
        
        recs=self.model.recommendForAllUsers(15).toPandas()
        nrecs=recs.recommendations.apply(pd.Series) \
                    .merge(recs, right_index = True, left_index = True) \
                    .drop(["recommendations"], axis = 1) \
                    .melt(id_vars = ['line_item_id_index'], value_name = "recommendation") \
                    .drop("variable", axis = 1) \
                    .dropna() 
        nrecs=nrecs.sort_values('line_item_id_index')
        nrecs=pd.concat([nrecs['recommendation'].apply(pd.Series), nrecs['line_item_id_index']], axis = 1)
        nrecs.columns = [
                'kw_id_index',
                'Rating',
                'line_item_id_index'
             ]
        nrecs['line_item_id']=nrecs['line_item_id_index'].map(d_lin_item)
        nrecs['kw_id']=nrecs['kw_id_index'].map(d_kw)
        nrecs=nrecs.sort_values('line_item_id')
        nrecs.reset_index(drop=True, inplace=True)
        self.new_kw=nrecs[['line_item_id','kw_id','Rating']]

        self.new_kw['User_reach']=self.new_kw['kw_id'].map(d_reach)
        self.df_tt_exp['preprocessed_kw']=self.df_tt_exp['keywords'].map(preprocessed_kw)
        print('Training is finished')
        print('--'*70)
        print('--'*70)
        return
    def recommend_kw(self,line_item_id,average_freq_cap,number_of_impression):
        existed_kw=list(self.df_tt_exp.loc[self.df_tt_exp['line_item_id']==line_item_id]['preprocessed_kw'].values)

        reco_camp_id=self.new_kw[(self.new_kw['line_item_id']==line_item_id)].sort_values('User_reach',ascending=False).reset_index()[['line_item_id','kw_id','Rating','User_reach']]

        reco_camp_id=reco_camp_id[~reco_camp_id['kw_id'].isin(existed_kw)]
        reco_camp_id['impression_provided']=reco_camp_id['User_reach']*average_freq_cap

        reco_camp_id['cum_impression'] = reco_camp_id.impression_provided.cumsum()
        #reco_camp_id=reco_camp_id.loc[reco_camp_id['cum_impression'] <= number_of_impression]
        reco_camp_id=reco_camp_id.reset_index()[['line_item_id','kw_id','Rating','impression_provided','cum_impression']]
        idx=reco_camp_id.loc[reco_camp_id['cum_impression'] <= number_of_impression].index[-1]
        #print('idx: ',idx,number_of_impression)
        if reco_camp_id['cum_impression'][idx]==number_of_impression:
            return reco_camp_id.loc[:str(int(idx))]
        return reco_camp_id.loc[:str(int(idx)+1)] if reco_camp_id.shape[0]>idx else reco_camp_id.loc[:str(int(idx))]
    
    def targeting_tactic_reco_cold_kw(self):

#         if self.advertiser_id not in list(df_kw_exp['advertiser_id'].unique()):
#             print('No recommendation can be provided since Advertiser has not past campaigns history and also line item is new campaign with no data')
#             return

        print('Line item id has no historical data and thus recommending based on the advertiser past campaigns data')
        df_kwx,ur_kw,preprocessed_kw,d_lin_item,d_kw=Preprocess_data(self.df_tt_exp,self.user_reach).preprocessing_kw_cs()
        d_reach={ur_kw.index[i]:ur_kw['user_reach'][i] for i in range(ur_kw.shape[0])}
        (training,test)=df_kwx.randomSplit([0.8, 0.2])
        als=ALS(maxIter=5,regParam=0.09,rank=25,userCol="advertiser_id_index",itemCol="kw_id_index",ratingCol="rating",coldStartStrategy="drop",nonnegative=True)
        self.model_cs=als.fit(training)

        evaluator=RegressionEvaluator(metricName="rmse",labelCol="rating",predictionCol="prediction")
        predictions=self.model_cs.transform(test)
        rmse=evaluator.evaluate(predictions)
        print("RMSE="+str(rmse))

        recs=self.model_cs.recommendForAllUsers(15).toPandas()
        nrecs=recs.recommendations.apply(pd.Series) \
                    .merge(recs, right_index = True, left_index = True) \
                    .drop(["recommendations"], axis = 1) \
                    .melt(id_vars = ['advertiser_id_index'], value_name = "recommendation") \
                    .drop("variable", axis = 1) \
                    .dropna() 
        nrecs=nrecs.sort_values('advertiser_id_index')
        nrecs=pd.concat([nrecs['recommendation'].apply(pd.Series), nrecs['advertiser_id_index']], axis = 1)
        nrecs.columns = [
                'kw_id_index',
                'Rating',
                'advertiser_id_index'
             ]
        nrecs['advertiser_id']=nrecs['advertiser_id_index'].map(d_lin_item)
        nrecs['kw_id']=nrecs['kw_id_index'].map(d_kw)
        nrecs=nrecs.sort_values('advertiser_id')
        nrecs.reset_index(drop=True, inplace=True)
        self.new_kw_cs=nrecs[['advertiser_id','kw_id','Rating']]

        self.new_kw_cs['User_reach']=self.new_kw_cs['kw_id'].map(d_reach)
        self.df_tt_exp['preprocessed_kw']=self.df_tt_exp['keywords'].map(preprocessed_kw)
        print('Training is finished')
        print('--'*70)
        print('--'*70)
        return 
    def recommend_kw_cs(self,advertiser_id,average_freq_cap,number_of_impression):
        reco_camp_id=self.new_kw_cs[(self.new_kw_cs['advertiser_id']==advertiser_id)].sort_values('User_reach',ascending=False).reset_index()[['advertiser_id','kw_id','Rating','User_reach']]

        reco_camp_id['impression_provided']=reco_camp_id['User_reach']*average_freq_cap

        reco_camp_id['cum_impression'] = reco_camp_id.impression_provided.cumsum()
        #reco_camp_id=reco_camp_id.loc[reco_camp_id['cum_impression'] <= number_of_impression]

        reco_camp_id=reco_camp_id.reset_index()[['advertiser_id','kw_id','Rating','impression_provided','cum_impression']]
        idx=reco_camp_id.loc[reco_camp_id['cum_impression'] <= number_of_impression].index[-1]
        if reco_camp_id['cum_impression'][idx]==number_of_impression:
            return reco_camp_id.loc[:str(int(idx))]
        return reco_camp_id.loc[:str(int(idx)+1)] if reco_camp_id.shape[0]>idx else reco_camp_id.loc[:str(int(idx))]
    def recommendation_kw(self,line_item_id,advertiser_id,average_freq_cap=1,number_of_impression=20000):
        if advertiser_id not in list(self.df_tt_exp['advertiser_id'].unique()):
            print('No recommendation can be provided since Advertiser has not past campaigns history and also line item is new campaign with no data')
            return
        if line_item_id not in list(self.df_tt_exp['line_item_id'].unique()):
            if (not self.is_cold_started_trained_already):
                self.fit(line_item_id,is_needed=True)
            return self.recommend_kw_cs(advertiser_id,average_freq_cap,number_of_impression)
        
        return self.recommend_kw(line_item_id,average_freq_cap,number_of_impression)


In [117]:
class targeting_tactic_model_BT:
    def __init__(self,df_tt_exp,user_reach_tt,camp_category='food'):
        self.df_tt_exp=df_tt_exp
        self.user_reach=user_reach_tt
        self.camp_category=camp_category
        self.is_cold_started_trained_already=False
        self.is_base_model_trained=False
    
    def fit(self,line_item_id,is_needed=False):
        if not self.is_base_model_trained and line_item_id in list(self.df_tt_exp['line_item_id'].unique()) :
            self.targeting_tactic_reco_us()
            self.is_base_model_trained=True
        if (not self.is_cold_started_trained_already) and is_needed:
            self.targeting_tactic_reco_cold_us()
            self.is_cold_started_trained_already=True
    def targeting_tactic_reco_us(self):
        df_us_exp=self.df_tt_exp.loc[self.df_tt_exp['camp_category']==self.camp_category]
        # if self.line_item_id not in list(self.df_tt_exp['line_item_id'].unique()):
        #     return self.targeting_tactic_reco_cold_us()

        df_usx,ur_us,d_lin_item,d_us=Preprocess_data(self.df_tt_exp,self.user_reach).preprocessing_us()
        d_reach={ur_us['user_segment'][i]:ur_us['user_reach'][i] for i in range(ur_us.shape[0])}

        (training,test)=df_usx.randomSplit([0.8, 0.2])
        als=ALS(maxIter=5,regParam=0.09,rank=25,userCol="line_item_id_index",itemCol="segments_index",ratingCol="rating",coldStartStrategy="drop",nonnegative=True)
        self.model=als.fit(training)

        evaluator=RegressionEvaluator(metricName="rmse",labelCol="rating",predictionCol="prediction")
        predictions=self.model.transform(test)
        rmse=evaluator.evaluate(predictions)
        print("RMSE="+str(rmse))

        recs=self.model.recommendForAllUsers(15).toPandas()
        nrecs=recs.recommendations.apply(pd.Series) \
                    .merge(recs, right_index = True, left_index = True) \
                    .drop(["recommendations"], axis = 1) \
                    .melt(id_vars = ['line_item_id_index'], value_name = "recommendation") \
                    .drop("variable", axis = 1) \
                    .dropna() 
        nrecs=nrecs.sort_values('line_item_id_index')
        nrecs=pd.concat([nrecs['recommendation'].apply(pd.Series), nrecs['line_item_id_index']], axis = 1)
        nrecs.columns = [
                'segments_index',
                'Rating',
                'line_item_id_index'
             ]
        nrecs['line_item_id']=nrecs['line_item_id_index'].map(d_lin_item)
        nrecs['segments']=nrecs['segments_index'].map(d_us)
        nrecs=nrecs.sort_values('line_item_id')
        nrecs.reset_index(drop=True, inplace=True)
        self.new_us=nrecs[['line_item_id','segments','Rating']]

        self.new_us['User_reach']=self.new_us['segments'].map(d_reach)
        print('Training is finished')
        print('--'*70)
        print('--'*70)
        return

    def recommend_us(self,line_item_id,average_freq_cap,number_of_impression):

        existed_us=list(self.df_tt_exp.loc[self.df_tt_exp['line_item_id']==line_item_id]['segments'].values)

        reco_camp_id=self.new_us[(self.new_us['line_item_id']==line_item_id)].sort_values('User_reach',ascending=False).reset_index()[['line_item_id','segments','Rating','User_reach']]

        reco_camp_id=reco_camp_id[~reco_camp_id['segments'].isin(existed_us)]
        reco_camp_id['impression_provided']=reco_camp_id['User_reach']*average_freq_cap

        reco_camp_id['cum_impression'] = reco_camp_id.impression_provided.cumsum()
        #reco_camp_id=reco_camp_id.loc[reco_camp_id['cum_impression'] <= number_of_impression]

        reco_camp_id=reco_camp_id.reset_index()[['line_item_id','segments','Rating','impression_provided','cum_impression']]
        idx=reco_camp_id.loc[reco_camp_id['cum_impression'] <= number_of_impression].index[-1]
        if reco_camp_id['cum_impression'][idx]==number_of_impression:
            return reco_camp_id.loc[:str(int(idx))]
        return reco_camp_id.loc[:str(int(idx)+1)] if reco_camp_id.shape[0]>idx else reco_camp_id.loc[:str(int(idx))]

    def targeting_tactic_reco_cold_us(self):
  
        # if self.advertiser_id not in list(self.df_tt_exp['advertiser_id'].unique()):
        #     print('No recommendation can be provided since Advertiser has not past campaigns history and also line item is new campaign with no data')
        #     return

        print('Line item has no historical data and thus recommending based on the advertiser past campaigns data')
        df_usx,ur_us,d_lin_item,d_us=Preprocess_data(self.df_tt_exp,self.user_reach).preprocessing_us_cs()
    #     d_reach={ur_us.index[i]:ur_us['user_reach'][i] for i in range(ur_us.shape[0])}
        d_reach={ur_us['user_segment'][i]:ur_us['user_reach'][i] for i in range(ur_us.shape[0])}
        (training,test)=df_usx.randomSplit([0.8, 0.2])
        als=ALS(maxIter=5,regParam=0.09,rank=25,userCol="advertiser_id_index",itemCol="segments_index",ratingCol="rating",coldStartStrategy="drop",nonnegative=True)
        self.model_cs=als.fit(training)

        evaluator=RegressionEvaluator(metricName="rmse",labelCol="rating",predictionCol="prediction")
        predictions=self.model_cs.transform(test)
        rmse=evaluator.evaluate(predictions)
        print("RMSE="+str(rmse))

        recs=self.model_cs.recommendForAllUsers(15).toPandas()
        nrecs=recs.recommendations.apply(pd.Series) \
                    .merge(recs, right_index = True, left_index = True) \
                    .drop(["recommendations"], axis = 1) \
                    .melt(id_vars = ['advertiser_id_index'], value_name = "recommendation") \
                    .drop("variable", axis = 1) \
                    .dropna() 
        nrecs=nrecs.sort_values('advertiser_id_index')
        nrecs=pd.concat([nrecs['recommendation'].apply(pd.Series), nrecs['advertiser_id_index']], axis = 1)
        nrecs.columns = [
                'segments_index',
                'Rating',
                'advertiser_id_index'
             ]
        nrecs['advertiser_id']=nrecs['advertiser_id_index'].map(d_lin_item)
        nrecs['segments']=nrecs['segments_index'].map(d_us)
        nrecs=nrecs.sort_values('advertiser_id')
        nrecs.reset_index(drop=True, inplace=True)
        self.new_us_cs=nrecs[['advertiser_id','segments','Rating']]

        self.new_us_cs['User_reach']=self.new_us_cs['segments'].map(d_reach)   
        print('Training is finished')
        print('--'*70)
        print('--'*70)
        return
    def recommend_us_cs(self,advertiser_id,average_freq_cap,number_of_impression):
        reco_camp_id=self.new_us_cs[(self.new_us_cs['advertiser_id']==advertiser_id)].sort_values('User_reach',ascending=False).reset_index()[['advertiser_id','segments','Rating','User_reach']]
        reco_camp_id['impression_provided']=reco_camp_id['User_reach']*average_freq_cap

        reco_camp_id['cum_impression'] = reco_camp_id.impression_provided.cumsum()
        #reco_camp_id=reco_camp_id.loc[reco_camp_id['cum_impression'] <= number_of_impression]

        reco_camp_id=reco_camp_id.reset_index()[['advertiser_id','segments','Rating','impression_provided','cum_impression']]
        idx=reco_camp_id.loc[reco_camp_id['cum_impression'] <= number_of_impression].index[-1]
        if reco_camp_id['cum_impression'][idx]==number_of_impression:
            return reco_camp_id.loc[:str(int(idx))]
        return reco_camp_id.loc[:str(int(idx)+1)] if reco_camp_id.shape[0]>idx else reco_camp_id.loc[:str(int(idx))]
    def recommendation_kw(self,line_item_id,advertiser_id,average_freq_cap=1,number_of_impression=20000):
        if advertiser_id not in list(self.df_tt_exp['advertiser_id'].unique()):
            print('No recommendation can be provided since Advertiser has not past campaigns history and also line item is new campaign with no data')
            return
        if line_item_id not in list(self.df_tt_exp['line_item_id'].unique()):
            if (not self.is_cold_started_trained_already):
                self.fit(line_item_id,is_needed=True)
            return self.recommend_us_cs(advertiser_id,average_freq_cap,number_of_impression)
        
        return self.recommend_us(line_item_id,average_freq_cap,number_of_impression)
    

Lets try to recommend keywords for KT targeting for the line item id: x0 and required number of impression= 20k for the advertiser id: 2 with campaign category=food

In [118]:
rex0=targeting_tactic_model_KT(df_kw_exp,user_reach_KW)
rex0.fit(line_item_id='x0')
rex1=rex0.recommendation_kw(line_item_id='x0',advertiser_id=2,number_of_impression=20000)
rex1

/opt/conda/anaconda/lib/python3.6/site-packages/ipykernel_launcher.py:35: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


RMSE=0.5171789747501693
Training is finished
--------------------------------------------------------------------------------------------------------------------------------------------
--------------------------------------------------------------------------------------------------------------------------------------------


/opt/conda/anaconda/lib/python3.6/site-packages/ipykernel_launcher.py:58: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/opt/conda/anaconda/lib/python3.6/site-packages/ipykernel_launcher.py:59: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


,line_item_id,kw_id,Rating,impression_provided,cum_impression
0,x0,p,0.820742,6492,6492
1,x0,yyp,0.753328,4935,11427
2,x0,w yp,0.797844,4584,16011
3,x0,wpy,0.795800,3644,19655
4,x0,wwpw,0.795800,3114,22769


Lets try to recommend keywords for KT targeting for the line item id: x31  which is a cold start problem and required number of impression= 20k for the advertiser id: 1 with campaign category=food

In [119]:
rex0=targeting_tactic_model_KT(df_kw_exp,user_reach_KW)
rex0.fit(line_item_id='x31')
rex1=rex0.recommendation_kw(line_item_id='x31',advertiser_id=2,number_of_impression=20000)
rex1

Line item id has no historical data and thus recommending based on the advertiser past campaigns data
RMSE=0.3775209226340664
Training is finished
--------------------------------------------------------------------------------------------------------------------------------------------
--------------------------------------------------------------------------------------------------------------------------------------------


/opt/conda/anaconda/lib/python3.6/site-packages/ipykernel_launcher.py:119: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


,advertiser_id,kw_id,Rating,impression_provided,cum_impression
0,2,y,0.828742,10301,10301
1,2,ywy,0.877638,7509,17810
2,2,py,0.909496,6780,24590


Lets try to recommend keywords for BT targeting for the line item id: x0 and required number of impression= 20k for the advertiser id: 2 with campaign category=food

In [120]:
rex0=targeting_tactic_model_BT(df_us_exp,user_reach_US)
rex0.fit(line_item_id='x0')
rex1=rex0.recommendation_kw(line_item_id='x0',advertiser_id=2,number_of_impression=20000)
rex1

RMSE=0.7496157406069953
Training is finished
--------------------------------------------------------------------------------------------------------------------------------------------
--------------------------------------------------------------------------------------------------------------------------------------------


/opt/conda/anaconda/lib/python3.6/site-packages/ipykernel_launcher.py:53: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


,line_item_id,segments,Rating,impression_provided,cum_impression
0,x0,segment43,0.831667,4768,4768
1,x0,segment26,0.462506,4744,9512
2,x0,segment39,0.450066,4300,13812
3,x0,segment45,0.741603,4103,17915
4,x0,segment13,0.848078,3858,21773


Lets try to recommend keywords for BT targeting for the line item id: x100  which is a cold start problem and required number of impression= 20k for the advertiser id: 1 with campaign category=food

In [121]:
rex0=targeting_tactic_model_BT(df_us_exp,user_reach_US)
rex0.fit(line_item_id='x100')
rex1=rex0.recommendation_kw(line_item_id='x00',advertiser_id=1,number_of_impression=20000)
rex1

Line item has no historical data and thus recommending based on the advertiser past campaigns data
RMSE=0.2969085315667031
Training is finished
--------------------------------------------------------------------------------------------------------------------------------------------
--------------------------------------------------------------------------------------------------------------------------------------------


/opt/conda/anaconda/lib/python3.6/site-packages/ipykernel_launcher.py:116: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


,advertiser_id,segments,Rating,impression_provided,cum_impression
0,1,segment46,0.870867,4880,4880
1,1,segment27,0.849177,4774,9654
2,1,segment43,0.922648,4768,14422
3,1,segment5,0.870867,4623,19045
4,1,segment39,0.870867,4300,23345
